# Definition of a simple 1-gram Model

In [1]:
# packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np

import os

loading the data

In [2]:
path = '/home/lorenz/Documents/Mathematik/24 FS/Semester_Paper_DCML/data/ABC/harmonies/n10op74_01.harmonies.tsv'
df_all = pd.read_csv(path, sep='\t')
print(df_all.columns)

# define new dataframe df with only the 'numeral' column
df = df_all['numeral']
# df = df_all['chord']
df_ = pd.factorize(df)
print(len(df_[1]))

Index(['mc', 'mn', 'quarterbeats', 'quarterbeats_all_endings', 'duration_qb',
       'mc_onset', 'mn_onset', 'timesig', 'staff', 'voice', 'label',
       'alt_label', 'globalkey', 'localkey', 'pedal', 'chord', 'numeral',
       'form', 'figbass', 'changes', 'relativeroot', 'cadence', 'phraseend',
       'chord_type', 'globalkey_is_minor', 'localkey_is_minor', 'chord_tones',
       'added_tones', 'root', 'bass_note'],
      dtype='object')
16


Markov chain model

In [3]:
class MarkovChain:
    def __init__(self, df):
        self.df = df
        self.df_ = pd.factorize(df)
        self.trans_mat = np.zeros((len(df_[1]), len(df_[1])))
    
    def clean_data(self):
        # if df_ contains a None value, remove it and the corresponding entry in df
        if None in self.df_[1]:
            ind = np.argwhere(self.df_[1] is None)
            self.df = np.delete(self.df, ind)
            self.df_ = pd.factorize(self.df)

    # calculate the transition matrix
    def markov_transition_matrix(self):
        trans_mat = np.zeros((len(self.df_[1]), len(self.df_[1])))

        # count of transitions
        for ch1 in self.df_[1]:
            inds = np.argwhere(self.df == ch1).flatten()
            # if first entry = 0, remove it 
            if inds[0] == 0:
                inds = np.delete(inds, 0)
            for ch2 in self.df_[1]:
                before = sum([chd == ch2 for chd in self.df[inds-1]])
                trans_mat[self.df_[1] == ch1, self.df_[1] == ch2] = before/len(self.df[inds])
        self.trans_mat = trans_mat
        return trans_mat

    def transform_chords_to_vectors(self, ch):
        assert(ch in self.df_[1])
        ind =  np.argwhere(self.df_[1] == ch)[0]
        vec = np.zeros(len(self.df_[1]))
        vec[ind] = 1
        return vec

    def fit(self):
        self.clean_data()
        self.trans_mat = self.markov_transition_matrix()
        return self

    def step(self, ch):
        # print('step form ', ch)
        vec = self.transform_chords_to_vectors(ch)
        out = np.dot(self.trans_mat.T, vec)
        return np.random.choice(self.df_[1], 1, p=out)[0]

    def predict(self, ch, n=3, verbose=False, start_at_current=False):
        seq = []
        if start_at_current:
            seq.append(ch)

        for i in range(n):
            ch = self.step(ch)
            seq.append(ch)
            if verbose:
                print(ch)
        return seq

fit the model

In [4]:
markov = MarkovChain(df).fit()
# markov.predict('I', n=36, verbose=True)

test the metrics

In [7]:
import chord_eval as ce

start = 'I'
length = 32

pred_chords = markov.predict(start, n=length, start_at_current=True)

org_chords = df.values[:length]

for y, y_hat in zip(pred_chords, org_chords):
    print(y, y_hat)
    tone_by_tone = ce.get_distance(y, y_hat)

['__new__', '__repr__', '__hash__', '__str__', '__getattribute__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__iter__', '__mod__', '__rmod__', '__len__', '__getitem__', '__add__', '__mul__', '__rmul__', '__contains__', 'encode', 'replace', 'split', 'rsplit', 'join', 'capitalize', 'casefold', 'title', 'center', 'count', 'expandtabs', 'find', 'partition', 'index', 'ljust', 'lower', 'lstrip', 'rfind', 'rindex', 'rjust', 'rstrip', 'rpartition', 'splitlines', 'strip', 'swapcase', 'translate', 'upper', 'startswith', 'endswith', 'removeprefix', 'removesuffix', 'isascii', 'islower', 'isupper', 'istitle', 'isspace', 'isdecimal', 'isdigit', 'isnumeric', 'isalpha', 'isalnum', 'isidentifier', 'isprintable', 'zfill', 'format', 'format_map', '__format__', 'maketrans', '__sizeof__', '__getnewargs__', '__doc__', '__setattr__', '__delattr__', '__init__', '__reduce_ex__', '__reduce__', '__getstate__', '__subclasshook__', '__init_subclass__', '__dir__', '__class__']
I I


AttributeError: module 'chord_eval' has no attribute 'get_distance'